In [56]:
# Import dependecies
import pandas as pd
import numpy as np
import difflib
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
movies_df = pd.read_csv('../data/movies.csv')
movies_df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [58]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [59]:
movies_df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [60]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director', 'production_companies']

In [61]:
# Replacing null values with null string
for feature in selected_features:
    movies_df[feature] = movies_df[feature].fillna(' ')


In [62]:
# combining all the selected_features
combined_df = movies_df['genres'] + movies_df['keywords'] + movies_df['tagline'] + movies_df['cast'] + movies_df['director'] + movies_df['production_companies']
print(combined_df)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy Romance A newlywed couple's honeymoon i...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801      A New Yorker in ShanghaiDaniel Henney Eliza ...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [63]:
vectorizer = TfidfVectorizer()

In [64]:
feature_vectors = vectorizer.fit_transform(combined_df)
print(feature_vectors)

  (0, 5298)	0.06645246096553001
  (0, 5491)	0.07719019984006875
  (0, 14533)	0.09975376128891036
  (0, 28659)	0.08441273909866218
  (0, 14913)	0.23262398777911492
  (0, 9949)	0.1851736772053612
  (0, 15861)	0.13809375760029433
  (0, 29869)	0.2842279275229562
  (0, 33710)	0.11027317639960828
  (0, 10196)	0.20352996305496868
  (0, 29712)	0.23262398777911492
  (0, 31296)	0.05957353841849757
  (0, 34573)	0.10540139206226319
  (0, 24635)	0.07188613686467524
  (0, 25101)	0.18109137834852304
  (0, 28315)	0.13210985968404387
  (0, 34597)	0.1994476641981305
  (0, 35012)	0.16573632196474855
  (0, 28279)	0.1830539443932836
  (0, 29357)	0.17164572609752055
  (0, 33929)	0.16681739135575374
  (0, 30250)	0.13594534852925405
  (0, 20611)	0.19000201194712799
  (0, 22904)	0.13152133496269175
  (0, 27531)	0.23262398777911492
  :	:
  (4802, 23971)	0.05863353513717311
  (4802, 18578)	0.05863353513717311
  (4802, 13831)	0.06343843322664075
  (4802, 15188)	0.05900347930588842
  (4802, 13117)	0.14760314786697

In [65]:
# Cosine Similarity: getting the similarity confidence value
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06833049 0.03197855 ... 0.02197666 0.         0.02118696]
 [0.06833049 1.         0.03877183 ... 0.02167348 0.         0.01247379]
 [0.03197855 0.03877183 1.         ... 0.01378982 0.         0.01073568]
 ...
 [0.02197666 0.02167348 0.01378982 ... 1.         0.         0.03202673]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.02118696 0.01247379 0.01073568 ... 0.03202673 0.         1.        ]]


In [66]:
# Getting input from user
movie_name = 'avatar'

In [67]:
movie_name

'avatar'

In [68]:
# Create a list with all the list of the movie names given in the dataset
list_of_all_titles = movies_df['title'].tolist()

In [69]:
find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_matches

['Avatar']

In [70]:
close_match = find_close_matches[0]
close_match

'Avatar'

In [71]:
idx_of_movie = movies_df[movies_df.title == close_match].index[0]
idx_of_movie

0

In [72]:
similarity_score = list(enumerate(similarity[0]))
similarity_score

[(0, 1.0000000000000004),
 (1, 0.06833048717548626),
 (2, 0.031978553516259375),
 (3, 0.04154310273623828),
 (4, 0.08591845784751687),
 (5, 0.041518749162116134),
 (6, 0.020226521131758206),
 (7, 0.03696833483996939),
 (8, 0.039289619683504584),
 (9, 0.07729816031358819),
 (10, 0.07060626559208848),
 (11, 0.017956980122591476),
 (12, 0.036553354419290286),
 (13, 0.04417082429386973),
 (14, 0.0779826876881534),
 (15, 0.04225512746348501),
 (16, 0.025705483717375292),
 (17, 0.030157228758691274),
 (18, 0.05404384456950005),
 (19, 0.04798207842047304),
 (20, 0.042461335099733384),
 (21, 0.03657537934420518),
 (22, 0.039505663149762095),
 (23, 0.12440025909845895),
 (24, 0.07614017458570654),
 (25, 0.20676142387565852),
 (26, 0.06343068268854211),
 (27, 0.04916274394230827),
 (28, 0.05458765580840474),
 (29, 0.020957835689382507),
 (30, 0.04102204385730837),
 (31, 0.047388447692332636),
 (32, 0.049297146820352945),
 (33, 0.20692696182934928),
 (34, 0.014264745502543185),
 (35, 0.0658682759

In [73]:
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[-1], reverse=True)
sorted_similar_movies

[(0, 1.0000000000000004),
 (101, 0.26246992864014085),
 (335, 0.26223524936025044),
 (1652, 0.2616387419486294),
 (3158, 0.259040646770482),
 (843, 0.24970971313222926),
 (292, 0.2412204054514432),
 (215, 0.23987856385137712),
 (261, 0.2252632776500491),
 (122, 0.21904586091081674),
 (192, 0.21057776373020787),
 (33, 0.20692696182934928),
 (25, 0.20676142387565852),
 (1124, 0.20340922055043578),
 (587, 0.19860811825988514),
 (838, 0.19743137940833652),
 (341, 0.189838416034043),
 (2403, 0.18859898966205746),
 (1275, 0.1876579869862649),
 (588, 0.18743837148299528),
 (282, 0.1840811522960645),
 (216, 0.18375684023076294),
 (942, 0.17898584795107278),
 (1834, 0.17882943915188768),
 (243, 0.17785317944781942),
 (835, 0.17307370198895983),
 (251, 0.17167733203821467),
 (1213, 0.16847249473519124),
 (1366, 0.16221096306544094),
 (278, 0.16170845205593615),
 (2824, 0.15976056754977777),
 (94, 0.1592494409068178),
 (539, 0.15667149665780164),
 (1990, 0.15533419452330122),
 (121, 0.15473749446

In [74]:
print('Movies suggested for you: \n')

i = 1
for i in range(30):
    index = sorted_similar_movies[i][0]
    title_from_index = movies_df[movies_df.index == index]['title'].values[0]
    print(f'{i} : {title_from_index}')

Movies suggested for you: 

0 : Avatar
1 : X-Men: First Class
2 : Rise of the Planet of the Apes
3 : Dragonball Evolution
4 : Alien
5 : The Watch
6 : Eragon
7 : Fantastic 4: Rise of the Silver Surfer
8 : Live Free or Die Hard
9 : X-Men Origins: Wolverine
10 : Australia
11 : X-Men: The Last Stand
12 : Titanic
13 : The Heat
14 : The Abyss
15 : Alien³
16 : Percy Jackson & the Olympians: The Lightning Thief
17 : Aliens
18 : Sunshine
19 : Wall Street: Money Never Sleeps
20 : True Lies
21 : Life of Pi
22 : The Book of Life
23 : Parental Guidance
24 : Night at the Museum
25 : Mr. Popper's Penguins
26 : Gulliver's Travels
27 : Aliens vs Predator: Requiem
28 : The Devil Wears Prada
29 : Planet of the Apes


In [75]:
# Single Function
def recommend_movie(movie_name):
    list_of_all_titles = movies_df['title'].tolist()
    find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
    close_match = find_close_matches[0]
    idx_of_movie = movies_df[movies_df.title == close_match]['index'].values[0]

    similarity_score = list(enumerate(similarity[idx_of_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    print('Movies suggested for you: \n')

    i = 1
    for i in range(30):
        index = sorted_similar_movies[i][0]
        title_from_index = movies_df[movies_df.index == index]['title'].values[0]
        print(f'{i} : {title_from_index}')


recommend_movie(movie_name = input('Enter your favorite movie name: '))
    

Movies suggested for you: 

0 : Three
1 : Run Lola Run
2 : Cloud Atlas
3 : The Baader Meinhof Complex
4 : Downfall
5 : The International
6 : Manderlay
7 : Good bye, Lenin!
8 : Dancer in the Dark
9 : Das Boot
10 : Amour
11 : Imagine Me & You
12 : The White Ribbon
13 : Mongol: The Rise of Genghis Khan
14 : Winged Migration
15 : The NeverEnding Story
16 : Headhunters
17 : Ghost Dog: The Way of the Samurai
18 : The Lives of Others
19 : Ajami
20 : Rubber
21 : Flame & Citron
22 : Black Book
23 : Bridge of Spies
24 : Carlos
25 : Perfume: The Story of a Murderer
26 : Killer Elite
27 : Cheri
28 : Ironclad
29 : Under the Skin
